In [92]:
import numpy as np
import pandas as pd
import pickle
import math
import pandas as pd
from pandas import HDFStore 
import os

###################
#nth_case=2
nth_case_list=np.arange(2,78)
nth_case_list=nth_case_list[nth_case_list!=39]
nth_case_list=nth_case_list[nth_case_list!=40]
n_f=7
#n_step=205
n_node=106
###################


###################################################################################
#node signal
_node_signal_list=[]
for nth_case in nth_case_list:

    node_signal_filename = 'data/data_74/'+str(nth_case)+'.txt'

    with open(node_signal_filename) as f:
            _node_signal = f.read().strip()
            _node_signal = _node_signal.replace('\n', ' ')
            _node_signal = _node_signal.split(' ')
    _node_signal=[x for x in _node_signal if ((len (x) > 10) )]
    _node_signal=_node_signal[9:-2]
    _node_signal=[x for x in _node_signal if ((len (x) < 30))]
    _node_signal=np.array(_node_signal).astype('float32')
    _node_signal=_node_signal.reshape((-1,n_node,n_f))
    n_step=_node_signal.shape[0]

    print("===========================_node_signal",_node_signal.shape)
    for i in range(n_f):
        print(np.min(_node_signal[:,:,i]),np.max(_node_signal[:,:,i]))
        
    _node_signal_list.append(_node_signal)

===========================_node_signal (205, 106, 7)
-57.9204 7.20446
-2.48061 18.0647
-189.827 61.2686
-0.0101126 0.000637231
-0.00649226 0.0313685
-0.000222451 0.00215211
0.0 0.0
===========================_node_signal (137, 106, 7)
-71.0506 8.89451
-3.6466 21.2389
-221.028 76.0398
-0.0101481 0.000791036
-0.00835712 0.0212361
-0.000209824 0.00258131
0.0 0.0
===========================_node_signal (130, 106, 7)
-85.1059 10.1848
-4.22114 25.2749
-267.657 90.9238
-0.0106199 0.0012047
-0.0106938 0.0202702
-0.000256624 0.00349409
0.0 0.0
===========================_node_signal (159, 106, 7)
-107.988 12.6635
-7.44655 34.5703
-318.377 116.734
-0.0124495 0.00135245
-0.0143746 0.0197956
-0.000330088 0.00416984
0.0 0.0
===========================_node_signal (168, 106, 7)
-102.31 11.9818
-5.6822 35.642
-282.684 112.328
-0.0107909 0.00132301
-0.0143322 0.0151041
-0.000319547 0.00393887
0.0 0.0
===========================_node_signal (205, 106, 7)
-96.8295 10.9033
-7.39948 41.0296
-258.944 107.

===========================_node_signal (190, 106, 7)
-9.06084 57.2423
-29.3104 3.29679
-141.113 48.2699
-0.00055453 0.00622857
-0.00313664 0.0113778
-0.000368965 0.00230296
0.0 0.0
===========================_node_signal (232, 106, 7)
-23.9896 114.48
-41.8001 6.822
-314.479 91.3902
-0.00136494 0.0106912
-0.00608154 0.0189444
-0.000791159 0.0049079
0.0 0.0
===========================_node_signal (180, 106, 7)
-31.5875 122.463
-40.6599 10.9867
-339.132 93.754
-0.00094184 0.0121691
-0.00648136 0.0164075
-0.00105744 0.00438838
0.0 0.0
===========================_node_signal (135, 106, 7)
-35.7511 115.996
-28.8446 7.91346
-338.157 85.1525
-0.000958733 0.0113482
-0.0073971 0.014279
-0.00100894 0.0042079
0.0 0.0
===========================_node_signal (141, 106, 7)
-54.8749 138.556
-31.5563 18.6483
-418.634 98.3504
-0.000845519 0.0137628
-0.0096343 0.0157251
-0.00152863 0.00434433
0.0 0.0
===========================_node_signal (143, 106, 7)
-75.85 153.31
-28.2132 12.2992
-497.02 105.208
-0.

In [107]:

########################################
# 0 is the latest observed sample.
x_offsets = np.sort(
    # np.concatenate(([-week_size + 1, -day_size + 1], np.arange(-11, 1, 1)))
    np.concatenate((np.arange(-11, 1, 1),))
)
# Predict the next one hour
y_offsets = np.sort(np.arange(1, 13, 1))
output_dir="data/"
########################################


####################################################################################
#load npy data
#node_signal=np.load("data/node_signal.npy")
x_list=[]
y_list=[]
for node_signal in _node_signal_list:
    num_samples, num_nodes, n_features = node_signal.shape
    data_list=[node_signal]

    #getting x and y
    data = np.concatenate(data_list, axis=-1)
    # epoch_len = num_samples + min(x_offsets) - max(y_offsets)
    x, y = [], []
    # t is the index of the last observation.
    min_t = abs(min(x_offsets))
    max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
    for t in range(min_t, max_t):
        x_t = data[t + x_offsets, ...]
        y_t = data[t + y_offsets, ...]
        x.append(x_t)
        y.append(y_t)
    x = np.stack(x, axis=0)
    y = np.stack(y, axis=0)
    print("========x shape: ", x.shape, ", y shape: ", y.shape)
    x_list.append(x)
    y_list.append(y)
x=np.concatenate(x_list,axis=0)
y=np.concatenate(y_list,axis=0)
print(x.shape,y.shape)#(15418, 12, 106, 7) (15418, 12, 106, 7)

========x shape:  (182, 12, 106, 7) , y shape:  (182, 12, 106, 7)
========x shape:  (114, 12, 106, 7) , y shape:  (114, 12, 106, 7)
========x shape:  (107, 12, 106, 7) , y shape:  (107, 12, 106, 7)
========x shape:  (136, 12, 106, 7) , y shape:  (136, 12, 106, 7)
========x shape:  (145, 12, 106, 7) , y shape:  (145, 12, 106, 7)
========x shape:  (182, 12, 106, 7) , y shape:  (182, 12, 106, 7)
========x shape:  (207, 12, 106, 7) , y shape:  (207, 12, 106, 7)
========x shape:  (215, 12, 106, 7) , y shape:  (215, 12, 106, 7)
========x shape:  (977, 12, 106, 7) , y shape:  (977, 12, 106, 7)
========x shape:  (206, 12, 106, 7) , y shape:  (206, 12, 106, 7)
========x shape:  (167, 12, 106, 7) , y shape:  (167, 12, 106, 7)
========x shape:  (209, 12, 106, 7) , y shape:  (209, 12, 106, 7)
========x shape:  (157, 12, 106, 7) , y shape:  (157, 12, 106, 7)
========x shape:  (109, 12, 106, 7) , y shape:  (109, 12, 106, 7)
========x shape:  (118, 12, 106, 7) , y shape:  (118, 12, 106, 7)
========x 

In [109]:
####################################################################################
# Write the data into npz file.
# num_test = 6831, using the last 6831 examples as testing.
# for the rest: 7/8 is used for training, and 1/8 is used for validation.
num_samples = x.shape[0]
num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train

# train
x_train, y_train = x[:num_train], y[:num_train]
# val
x_val, y_val = (
    x[num_train: num_train + num_val],
    y[num_train: num_train + num_val],
)
# test
x_test, y_test = x[-num_test:], y[-num_test:]

print("train",x_train.shape,y_train.shape)
print("val",x_val.shape,y_val.shape)
print("test",x_test.shape,y_test.shape)

train (10793, 12, 106, 7) (10793, 12, 106, 7)
val (1541, 12, 106, 7) (1541, 12, 106, 7)
test (3084, 12, 106, 7) (3084, 12, 106, 7)


In [113]:
for cat in ["train", "val", "test"]:
    _x, _y = locals()["x_" + cat], locals()["y_" + cat]
    print(cat, "x: ", _x.shape, "y:", _y.shape)
    np.savez_compressed(
        os.path.join(output_dir, "%s.npz" % cat),
        x=_x,
        y=_y,
        x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
        y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
    )

train x:  (10793, 12, 106, 7) y: (10793, 12, 106, 7)
val x:  (1541, 12, 106, 7) y: (1541, 12, 106, 7)
test x:  (3084, 12, 106, 7) y: (3084, 12, 106, 7)
